In [0]:
# The code was removed by Watson Studio for sharing.

In [0]:
#dd-ignore

!pip install --user dd-scenario


In [0]:
#dd-ignore

from dd_scenario import *

# Creates a client...
# If you want to be able to call solve() on the client, you have to provide your API Key
# client = Client(pc=pc, apikey='IAM_APIKEY')
client = Client(pc=pc)


In [0]:
#dd-ignore

#Get 'PMDO' decision...
dd_model_builder = client.get_model_builder(name="PMDO")

#Get scenario 'Optimized MA'...
scenario = dd_model_builder.get_scenario(name="Optimized MA")

#Load all input data as a map { data_name: data_frame }
inputs = scenario.get_tables_data(category='input')
# This will hold all outputs as a map { data_name: data_frame }
outputs = {}

# we use a lock to access ``outputs``. This allows solves() to
# be aborted without race condition in data writting
import threading
output_lock = threading.Lock()



In [0]:
from docplex.mp.model import *
from docplex.mp.utils import *
from docplex.util.status import JobSolveStatus
from docplex.mp.conflict_refiner import ConflictRefiner, VarUbConstraintWrapper, VarLbConstraintWrapper
from docplex.mp.relaxer import Relaxer
import time
import sys
import operator

import pandas as pd
import numpy as np
import math

import codecs
import sys

# Handle output of unicode strings
if sys.version_info[0] < 3:
    sys.stdout = codecs.getwriter('utf8')(sys.stdout)


# Label constraint
def helper_add_labeled_cplex_constraint(mdl, expr, label, context=None, columns=None):
    global expr_counter
    if isinstance(expr, np.bool_):
        expr = expr.item()
    if isinstance(expr, bool):
        pass  # Adding a trivial constraint: if infeasible, docplex will raise an exception it is added to the model
    else:
        expr.name = '_L_EXPR_' + str(len(expr_to_info) + 1)
        if columns:
            ctxt = ", ".join(str(getattr(context, col)) for col in columns)
        else:
            if context:
                ctxt = context.Index if isinstance(context.Index, str) is not None else ", ".join(context.Index)
            else:
                ctxt = None
        expr_to_info[expr.name] = (label, ctxt)
    mdl.add(expr)

def helper_get_column_name_for_property(property):
    return helper_property_id_to_column_names_map.get(property, 'unknown')


def helper_get_index_names_for_type(dataframe, type):
    if not is_pandas_dataframe(dataframe):
        return None
    return [name for name in dataframe.index.names if name in helper_concept_id_to_index_names_map.get(type, [])]


helper_concept_id_to_index_names_map = {
    'machine': ['id_of_Machine'],
    'cAssignmentValueConcept': ['id_of_Predicted_failure'],
    'cResource': ['id_of_Day'],
    'day': ['id_of_Day'],
    'cActivity': ['id_of_Machine'],
    'predicted_failure': ['id_of_Predicted_failure']}
helper_property_id_to_column_names_map = {
    'cAssignmentValueConcept.value': 'failure',
    'predicted_failure.machine': 'machine',
    'predicted_failure.day': 'day',
    'cAssignmentValueConcept.resource': 'day',
    'machine.id': 'id',
    'predicted_failure.failure': 'failure',
    'cAssignmentValueConcept.activity': 'machine',
    'day.id': 'id'}


# Data model definition for each table
# Data collection: list_of_Day ['id']
# Data collection: list_of_Machine ['id']
# Data collection: list_of_Predicted_failure ['day', 'failure', 'machine', '__line']

# Create a pandas Dataframe for each data table
list_of_Day = inputs[u'day']
list_of_Day = list_of_Day[[u'id']].copy()
list_of_Day.rename(columns={u'id': 'id'}, inplace=True)
list_of_Machine = inputs[u'machine']
list_of_Machine = list_of_Machine[[u'id']].copy()
list_of_Machine.rename(columns={u'id': 'id'}, inplace=True)
list_of_Predicted_failure = inputs[u'predicted_failure']
list_of_Predicted_failure = list_of_Predicted_failure[[u'day', u'failure', u'machine']].copy()
list_of_Predicted_failure.rename(columns={u'day': 'day', u'failure': 'failure', u'machine': 'machine'}, inplace=True)

# Set index when a primary key is defined
list_of_Day.set_index('id', inplace=True)
list_of_Day.sort_index(inplace=True)
list_of_Day.index.name = 'id_of_Day'
list_of_Machine.set_index('id', inplace=True)
list_of_Machine.sort_index(inplace=True)
list_of_Machine.index.name = 'id_of_Machine'
list_of_Predicted_failure.index.name = 'id_of_Predicted_failure'


# Create data frame as cartesian product of: Day x Machine
list_of_ResourceAssignment = pd.DataFrame(index=pd.MultiIndex.from_product((list_of_Day.index, list_of_Machine.index), names=['id_of_Day', 'id_of_Machine']))




def build_model():
    mdl = Model()

    # Definition of model variables
    list_of_ResourceAssignment['resourceAssignmentVar'] = mdl.binary_var_list(len(list_of_ResourceAssignment))


    # Definition of model
    # Objective cMaximizeAssignmentsAutoSelected-
    # Combine weighted criteria: 
    # 	cMaximizeAssignmentsAutoSelected cMaximizeAssignmentsAutoSelected 1.2{
    # 	cMaximizeAssignments.assignment = cResourceAssignment[day, machine],
    # 	cScaledGoal.scaleFactorExpr = 1,
    # 	cSingleCriterionGoal.goalFilter = null,
    # 	cSingleCriterionGoal.numericExpr = decisionPath(cResourceAssignment[day, machine])} with weight 5.0
    # 	cMaximizeAssignmentValue cMaximizeAssignmentValue 1.2{
    # 	cMaximizeAssignmentValue.assignment = cResourceAssignment[day, machine],
    # 	cMaximizeAssignmentValue.assignmentValue = predicted_failure,
    # 	cScaledGoal.scaleFactorExpr = 1,
    # 	cSingleCriterionGoal.goalFilter = null,
    # 	cSingleCriterionGoal.numericExpr = total predicted_failure [join dimensions(cResourceAssignment[day, machine] / day, cResourceAssignment[day, machine] / machine) with dimensions(predicted_failure / day, predicted_failure / machine)] / failure over cResourceAssignment[day, machine]} with weight 5.0
    agg_ResourceAssignment_resourceAssignmentVar_SG1 = mdl.sum(list_of_ResourceAssignment.resourceAssignmentVar)
    join_ResourceAssignment_SG2 = list_of_ResourceAssignment.reset_index().merge(list_of_Predicted_failure.reset_index(), left_on=['id_of_Day', 'id_of_Machine'], right_on=['day', 'machine']).set_index(['id_of_Day', 'id_of_Machine', 'id_of_Predicted_failure'])
    join_Predicted_failure_SG2 = list_of_Predicted_failure.join(join_ResourceAssignment_SG2, rsuffix='_right', how='inner')
    reindexed_Predicted_failure_SG2 = join_Predicted_failure_SG2.reset_index().set_index(['id_of_Day', 'id_of_Machine'])
    join_ResourceAssignment_SG2_2 = list_of_ResourceAssignment.reset_index().merge(reindexed_Predicted_failure_SG2.reset_index(), left_on=['id_of_Day', 'id_of_Machine'], right_on=['id_of_Day', 'id_of_Machine'], suffixes=('', '_right')).set_index(['id_of_Day', 'id_of_Machine'])
    join_ResourceAssignment_SG2_2['conditioned_failure'] = join_ResourceAssignment_SG2_2.resourceAssignmentVar * join_ResourceAssignment_SG2_2.failure
    agg_ResourceAssignment_conditioned_failure_SG2 = mdl.sum(join_ResourceAssignment_SG2_2.conditioned_failure)
    
    kpis_expression_list = [
        (1, 16.0, agg_ResourceAssignment_resourceAssignmentVar_SG1, 1, 0, u'the number of day to machine assignments'),
        (1, 16.0, agg_ResourceAssignment_conditioned_failure_SG2, 1, 0, u'overall quality of day to machine assignments according to predicted_failures')]
    custom_code.update_goals_list(kpis_expression_list)
    
    for _, kpi_weight, kpi_expr, kpi_factor, kpi_offset, kpi_name in kpis_expression_list:
        mdl.add_kpi(kpi_weight * ((kpi_expr * kpi_factor) - kpi_offset), publish_name=kpi_name)
    
    mdl.maximize(sum([kpi_sign * kpi_weight * ((kpi_expr * kpi_factor) - kpi_offset) for kpi_sign, kpi_weight, kpi_expr, kpi_factor, kpi_offset, kpi_name in kpis_expression_list]))
    
    # [ST_1] Constraint : cLimitNumberOfResourcesAssignedToEachActivity_cIterativeRelationalConstraint
    # The number of days assigned to each machine is less than or equal to 1
    # Label: CT_1_The_number_of_days_assigned_to_each_machine_is_less_than_or_equal_to_1
    groupbyLevels = ['id_of_Machine']
    groupby_ResourceAssignment = list_of_ResourceAssignment.resourceAssignmentVar.groupby(level=groupbyLevels).sum().to_frame()
    for row in groupby_ResourceAssignment.itertuples(index=True):
        helper_add_labeled_cplex_constraint(mdl, row.resourceAssignmentVar <= 1, u'The number of days assigned to each machine is less than or equal to 1', row)
    
    # [ST_2] Constraint : cBasicLimitNumberOfActivitiesAssignedToEachResource_cIterativeRelationalConstraint
    # The number of machines assigned to each day is less than or equal to 2
    # Label: CT_2_The_number_of_machines_assigned_to_each_day_is_less_than_or_equal_to_2
    groupbyLevels = ['id_of_Day']
    groupby_ResourceAssignment = list_of_ResourceAssignment.resourceAssignmentVar.groupby(level=groupbyLevels).sum().to_frame()
    for row in groupby_ResourceAssignment.itertuples(index=True):
        helper_add_labeled_cplex_constraint(mdl, row.resourceAssignmentVar <= 2, u'The number of machines assigned to each day is less than or equal to 2', row)


    return mdl


def solve_model(mdl):
    mdl.parameters.timelimit = 120
    # Call to custom code to update parameters value
    custom_code.update_solver_params(mdl.parameters)
    # Update parameters value according to environment variables definition
    cplex_param_env_prefix = 'ma.cplex.'
    cplex_params = [name.qualified_name for name in mdl.parameters.generate_params()]
    for param in cplex_params:
        env_param = cplex_param_env_prefix + param
        param_value = get_environment().get_parameter(env_param)
        if param_value:
            # Updating parameter value
            print("Updated value for parameter %s = %s" % (param, param_value))
            parameters = mdl.parameters
            for p in param.split('.')[1:]:
                parameters = parameters.__getattribute__(p)
            parameters.set(param_value)

    msol = mdl.solve(log_output=True)
    if not msol:
        print("!!! Solve of the model fails")
        if mdl.get_solve_status() == JobSolveStatus.INFEASIBLE_SOLUTION or mdl.get_solve_status() == JobSolveStatus.INFEASIBLE_OR_UNBOUNDED_SOLUTION:
            crefiner = ConflictRefiner()
            conflicts = crefiner.refine_conflict(model, log_output=True)
            export_conflicts(conflicts)
            
    print('Solve status: %s' % mdl.get_solve_status())
    mdl.report()
    return msol


expr_to_info = {}


def export_conflicts(conflicts):
    # Display conflicts in console
    print('Conflict set:')
    list_of_conflicts = pd.DataFrame(columns=['constraint', 'context', 'detail'])
    for conflict, index in zip(conflicts, range(len(conflicts))):
        st = conflict.status
        ct = conflict.element
        label, context = expr_to_info.get(conflict.name, ('N/A', conflict.name))
        label_type = type(conflict.element)
        if isinstance(conflict.element, VarLbConstraintWrapper) \
                or isinstance(conflict.element, VarUbConstraintWrapper):
            label = 'Upper/lower bound conflict for variable: {}'.format(conflict.element._var)
            context = 'Decision variable definition'
            ct = conflict.element.get_constraint()

        # Print conflict information in console
        print("Conflict involving constraint: %s, \tfor: %s -> %s" % (label, context, ct))
        list_of_conflicts = list_of_conflicts.append({'constraint': label, 'context': str(context), 'detail': ct},
                                                     ignore_index=True)

    # Update of the ``outputs`` dict must take the 'Lock' to make this action atomic,
    # in case the job is aborted
    global output_lock
    with output_lock:
        outputs['list_of_conflicts'] = list_of_conflicts


def export_solution(msol):
    start_time = time.time()
    list_of_ResourceAssignment_solution = pd.DataFrame(index=list_of_ResourceAssignment.index)
    list_of_ResourceAssignment_solution['resourceAssignmentVar'] = msol.get_values(list_of_ResourceAssignment.resourceAssignmentVar.values)

    # Filter rows for non-selected assignments
    list_of_ResourceAssignment_solution = list_of_ResourceAssignment_solution[list_of_ResourceAssignment_solution.resourceAssignmentVar > 0.5]

    # Update of the ``outputs`` dict must take the 'Lock' to make this action atomic,
    # in case the job is aborted
    global output_lock
    with output_lock:
        outputs['list_of_ResourceAssignment_solution'] = list_of_ResourceAssignment_solution.reset_index()
        custom_code.post_process_solution(msol, outputs)

    elapsed_time = time.time() - start_time
    print('solution export done in ' + str(elapsed_time) + ' secs')
    return


# Instantiate CustomCode class if definition exists
try:
    custom_code = CustomCode(globals())
except NameError:
    # Create a dummy anonymous object for custom_code
    custom_code = type('', (object,), {'preprocess': (lambda *args: None),
                                       'update_goals_list': (lambda *args: None),
                                       'update_model': (lambda *args: None),
                                       'update_solver_params': (lambda *args: None),
                                       'post_process_solution': (lambda *args: None)})()

# Custom pre-process
custom_code.preprocess()

print('* building wado model')
start_time = time.time()
model = build_model()

# Model customization
custom_code.update_model(model)

elapsed_time = time.time() - start_time
print('model building done in ' + str(elapsed_time) + ' secs')

print('* running wado model')
start_time = time.time()
msol = solve_model(model)
elapsed_time = time.time() - start_time
print('model solve done in ' + str(elapsed_time) + ' secs')
if msol:
    export_solution(msol)